pip install scib
pip install tensorflow
pip install harmony-pytorch
conda install -c conda-forge python-annoy
pip install scanorama
pip install git+https://github.com/theislab/scgen.git

In [44]:
#Imports
import scib
import os
import scanpy as sc
import tensorflow as tf
import matplotlib
import anndata
from scipy import sparse
import pandas as pd

In [114]:
#---Load Data---
dataset_name = 'Lung_Atlas_public'
label_key = 'cell_type'
batch_key = 'batch'


#set base path to load data: goes back one directory and then into the data
base_path = os.path.join('..', 'data')

# read dataset into an anndata object:  Category - Cells of the brain
inPath = os.path.join(base_path, f"{dataset_name}.h5ad")
adata = sc.read(inPath)

In [115]:
adata_integrated = scib.integration.scanorama(adata, batch_key, hvg=None)

Found 15148 genes among all datasets
[[0.00000000e+00 4.33628319e-01 1.03982301e-01 5.64159292e-02
  4.15929204e-01 5.97345133e-02 2.54424779e-02 5.19911504e-02
  1.10619469e-03 6.63716814e-03 4.42477876e-03 6.63716814e-03
  9.81308411e-03 2.21238938e-03 3.31858407e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 6.47186147e-01 1.75050302e-01
  3.27967807e-01 3.22434608e-01 1.30081301e-02 8.04828974e-03
  1.00603622e-03 1.50905433e-02 7.48129676e-03 1.10663984e-02
  2.89719626e-02 2.01207243e-03 4.70957614e-03 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.64069264e-01
  7.48917749e-01 5.49783550e-01 3.46320346e-02 1.29870130e-02
  1.51515152e-02 2.59740260e-02 3.24675325e-02 4.32900433e-03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.80748214e-01 2.76640978e-01 1.24661247e-02 2.06327373e-03
  0.00000000e+00 2.83687943e-03 3.69908562e-02 6.64363540e-03
  0.00000000e+00 0.00000000e+0

In [119]:
print(adata_integrated.X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 490275526 stored elements and shape (32472, 15148)>
  Coords	Values
  (0, 0)	-0.0006595508845566153
  (0, 1)	-0.0020450527624798678
  (0, 2)	-0.0006356212792957765
  (0, 3)	0.01176976313564461
  (0, 4)	-0.00098563756214346
  (0, 5)	-0.0005108111202167678
  (0, 6)	-0.0009660484752144437
  (0, 7)	-0.00033785810210537745
  (0, 8)	0.002852583582224716
  (0, 9)	-0.0009249347163816271
  (0, 10)	0.016364982561392044
  (0, 11)	0.0002970869642423664
  (0, 12)	0.018698739555529798
  (0, 13)	0.0009284325796758885
  (0, 14)	0.0007973880584026777
  (0, 15)	-0.0013612029951225151
  (0, 16)	0.0007015657032004683
  (0, 17)	-0.00033197446850716364
  (0, 18)	0.0007107817408819043
  (0, 19)	0.00040985579066065266
  (0, 20)	-0.00028103277826865776
  (0, 21)	-0.0022219546908324537
  (0, 22)	0.0012488712788697816
  (0, 23)	3.726029368018098e-05
  (0, 24)	-0.001688767815670383
  :	:
  (32471, 15123)	3.479316201232428e-05
  (32471, 15124)	0.0005885

In [ ]:
def scoring_integration(    adata,                  #unintegrated Iput
                            integrated,             #integrated Input
                            label_key,              #Batchkey
                            batch_key,              #Labelkey
                            n_top_genes,             #True to enable Top 2000 varibel gene selection
                            embd                    #True to use embedded output
            ):
    if embd == True:
        sc.pp.neighbors(integrated, use_rep="X_emb")
    else:     
        #Run PCA
        if n_top_genes == True:
            scib.pp.reduce_data(
                integrated, n_top_genes=2000, batch_key="batch", pca=True, neighbors=True
            )
        elif n_top_genes == False:
            scib.pp.reduce_data(
                integrated, batch_key="batch", pca=True, neighbors=True
            )    
          
    #Cluster PCA results
    scib.me.cluster_optimal_resolution(integrated, cluster_key="cluster", label_key=label_key)

    #--Bio Scores--

    #Adjusted Rand Index
    ari = scib.me.ari(integrated, cluster_key="cluster", label_key=label_key)

    #HVG overlap
    hvg = scib.me.hvg_overlap(adata, integrated, batch_key=batch_key)

    #isolated_labels_asw
    asw = scib.me.isolated_labels_asw(integrated, label_key=label_key, batch_key=batch_key, embed="X_pca")

    #isolated_labels_f1
    f1 = scib.me.isolated_labels_f1(integrated, label_key=label_key, batch_key=batch_key, embed="X_pca")

    #Normalized mutual information
    nmi = scib.me.nmi(integrated, cluster_key="cluster", label_key=label_key)

    #Average silhouette width (ASW)
    sil= scib.me.silhouette(integrated, label_key=label_key, embed="X_pca")

    #--Cluster Scores--

    #Graph Connectivity
    graph = scib.me.graph_connectivity(integrated, label_key=label_key)

    #Principal component regression score
    pcr = scib.me.pcr_comparison(adata, integrated, covariate=batch_key)

    # Modified average silhouette width (ASW) of batch
    sil_batch = scib.me.silhouette_batch(integrated, batch_key=batch_key, label_key=label_key, embed="X_pca")

    avg_bio = (ari+hvg+asw+f1+nmi+sil)/6
    avg_batch =(graph+pcr+sil_batch)/3

    Scores = pd.DataFrame({
        'ari': ari,
        'hvg': hvg,
        'asw': asw,
        'f1': f1,
        'nmi': nmi,
        'sil': sil,
        'graph': graph,
        'pcr': pcr,
        'sil_batch': sil_batch,
        'avg_bio': avg_bio,
        'avg_batch': avg_batch
        }, index=[0])
    return Scores

In [ ]:
#Function including all integrators to test
def integrate(adata, batch_key, label_key, alg):
    if alg == 'None':
        return adata
    elif alg == 'combat':
        return sc.pp.combat(adata, batch_key)
    elif alg == 'harmony':
        return scib.integration.harmony(adata, batch_key, hvg=None)
    elif alg == 'scanorama':
        return scib.integration.scanorama(adata, batch_key, hvg=None)
    elif alg == 'scgen':
        return scib.integration.scgen(adata, batch_key, label_key, epochs=1, hvg=None)
     

In [53]:
#combat
integrated = integrate(adata, batch_key, label_key, 'combat')

c:\Users\lklei\anaconda3\envs\scib\lib\site-packages\scanpy\preprocessing\_combat.py:347: RuntimeWarning: divide by zero encountered in divide
  (abs(g_new - g_old) / g_old).max(), (abs(d_new - d_old) / d_old).max()


In [96]:
sc.pp.combat(integrated, batch_key)

In [97]:
print(integrated.X)

[[ 3.58622418e-03  4.15820410e-02  2.50539339e-03 ...  1.27390143e-01
   3.26949211e-02  3.46053263e-02]
 [ 3.58622418e-03  4.15820410e-02  2.50539339e-03 ...  1.27390143e-01
   3.26949211e-02  3.46053263e-02]
 [ 3.58622418e-03  4.15820410e-02  2.50539339e-03 ...  1.27390143e-01
   3.26949211e-02  3.46053263e-02]
 ...
 [-8.08593590e-03  2.13886412e-01 -9.72044522e-04 ...  2.30869960e-01
  -7.43148710e-03  9.79257292e-03]
 [-3.49565328e-03  1.86692052e-02  4.45518679e-05 ...  5.79512189e-01
  -6.62302241e-03  5.69640143e-03]
 [-1.31429719e-03 -5.22984959e-02  4.30749283e-04 ... -7.60761785e-02
   4.09955563e-03  1.75874232e-02]]


In [108]:
import warnings
#Ignore warnings
warnings.filterwarnings("ignore")

df =      scoring_integration(  adata,                  #unintegrated Iput
                                integrated,             #integrated Input
                                label_key,              #Batchkey
                                batch_key,              #Labelkey
                                True                    #True to enable Top 2000 varibel gene selection
            )

HVG
Using 1374 HVGs from full intersect set
Using 337 HVGs from n_batch-1 set
Using 119 HVGs from n_batch-2 set
Using 73 HVGs from n_batch-3 set
Using 37 HVGs from n_batch-4 set
Using 32 HVGs from n_batch-5 set
Using 28 HVGs from n_batch-6 set
Using 2000 HVGs
Computed 2000 highly variable genes
PCA
Nearest Neigbours
resolution: 0.2, nmi: 0.7147611954178497
resolution: 0.4, nmi: 0.6999043667409671
resolution: 0.6, nmi: 0.701037628776295
resolution: 0.8, nmi: 0.6955639987741002
resolution: 1.0, nmi: 0.6827542612366395
resolution: 1.2, nmi: 0.6740500062939119
resolution: 1.4, nmi: 0.6645620360280255
resolution: 1.6, nmi: 0.6626741649537158
resolution: 1.8, nmi: 0.6587239362501789
resolution: 2.0, nmi: 0.6540746698297745
optimised clustering against cell_type
optimal cluster resolution: 0.2
optimal score: 0.7147611954178497
isolated labels: no more than 4 batches per label
isolated labels: ['Type 1']
Type 1: 0.6163010001182556
isolated labels: no more than 4 batches per label
isolated labe

In [110]:
df

,ari,hvg,asw,f1,nmi,sil,graph,pcr,sil_batch,avg_bio,avg_batch
0,0.558804,0.4055,0.616301,0.719298,0.714761,0.557812,0.972707,0.999913,0.903505,0.595413,0.958709


In [113]:
print(integrated.X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 491885856 stored elements and shape (32472, 15148)>
  Coords	Values
  (0, 0)	0.003586224181421525
  (0, 1)	0.04158204104380849
  (0, 2)	0.002505393387202459
  (0, 3)	0.059741989534512395
  (0, 4)	0.02605183374844842
  (0, 5)	0.03118775416502366
  (0, 6)	0.009027894286726998
  (0, 7)	0.007262282922119213
  (0, 8)	0.07056265486106067
  (0, 9)	0.027284821497432814
  (0, 10)	0.13179890848627762
  (0, 11)	0.07459116217513262
  (0, 12)	0.09933111623134916
  (0, 13)	0.0166691510676617
  (0, 14)	0.0008635538160030494
  (0, 15)	0.04295258192671808
  (0, 16)	0.004803898142626935
  (0, 17)	0.021813739074439585
  (0, 18)	0.04164632549244773
  (0, 19)	0.05813598103694643
  (0, 20)	0.013259417749538292
  (0, 21)	0.05163320956415121
  (0, 22)	0.001300151530690831
  (0, 23)	0.016265578964892058
  (0, 24)	0.03281044372489625
  :	:
  (32471, 15123)	-0.00016157152311254645
  (32471, 15124)	0.0006181811973036445
  (32471, 15125)	0.0027752176444

In [116]:
new_row

,ari,hvg,asw,f1,nmi,sil,graph,pcr,sil_batch,avg_bio,avg_batch
0,0.558804,0.4195,0.611128,0.719437,0.714761,0.558601,0.971769,0.99991,0.90638,0.597039,0.959353


In [64]:
#harmony
integrated_1 = integrate(adata, batch_key, label_key, 'harmony')

	Initialization is completed.
	Completed 1 / 10 iteration(s).
	Completed 2 / 10 iteration(s).
	Completed 3 / 10 iteration(s).
	Completed 4 / 10 iteration(s).
	Completed 5 / 10 iteration(s).
	Completed 6 / 10 iteration(s).
	Completed 7 / 10 iteration(s).
	Completed 8 / 10 iteration(s).
	Completed 9 / 10 iteration(s).
	Completed 10 / 10 iteration(s).


In [79]:
print(integrated_1.X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 491885856 stored elements and shape (32472, 15148)>
  Coords	Values
  (0, 0)	0.00354392546535237
  (0, 1)	0.04139257226125962
  (0, 2)	0.002497130563915313
  (0, 3)	0.05973961721884444
  (0, 4)	0.026008626459940922
  (0, 5)	0.031153579865290652
  (0, 6)	0.008977139573235822
  (0, 7)	0.0070944598098099205
  (0, 8)	0.0704876524711607
  (0, 9)	0.027239567302821843
  (0, 10)	0.1318195269059124
  (0, 11)	0.07451306764010414
  (0, 12)	0.09925780114019216
  (0, 13)	0.016625332138273413
  (0, 14)	0.00042173567252741344
  (0, 15)	0.04290314552091645
  (0, 16)	0.00478193490068811
  (0, 17)	0.021780422530792525
  (0, 18)	0.04159451049795054
  (0, 19)	0.05805955618327777
  (0, 20)	0.013204736485861436
  (0, 21)	0.051548361023605195
  (0, 22)	0.0009919076310676723
  (0, 23)	0.016231701644647132
  (0, 24)	0.03272229915949447
  :	:
  (32471, 15123)	-0.00016161404517272973
  (32471, 15124)	0.0006237931066694746
  (32471, 15125)	0.0027828451

In [100]:
print(adata.X)

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 63423541 stored elements and shape (32472, 15148)>
  Coords	Values
  (0, 29)	1.2873256206512451
  (0, 299)	1.2873256206512451
  (0, 539)	1.2873256206512451
  (0, 704)	1.2873256206512451
  (0, 708)	1.2873256206512451
  (0, 709)	1.2873256206512451
  (0, 748)	1.2873256206512451
  (0, 900)	1.2873256206512451
  (0, 904)	1.2873256206512451
  (0, 923)	1.2873256206512451
  (0, 925)	1.2873256206512451
  (0, 941)	1.2873256206512451
  (0, 1021)	1.2873256206512451
  (0, 1084)	1.2873256206512451
  (0, 1135)	1.8319681882858276
  (0, 1170)	1.2873256206512451
  (0, 1194)	1.8319681882858276
  (0, 1253)	1.2873256206512451
  (0, 1270)	1.2873256206512451
  (0, 1389)	1.2873256206512451
  (0, 1622)	1.2873256206512451
  (0, 1650)	1.2873256206512451
  (0, 1672)	1.2873256206512451
  (0, 1768)	1.2873256206512451
  (0, 1787)	1.2873256206512451
  :	:
  (32471, 14614)	0.7971258163452148
  (32471, 14631)	1.0191913843154907
  (32471, 14632)	0.740645110607

In [ ]:
scib.integration.scanorama(adata, batch_key, hvg=None)
adata

Found 15148 genes among all datasets
[[0.00000000e+00 4.33628319e-01 1.03982301e-01 5.64159292e-02
  4.15929204e-01 5.97345133e-02 2.54424779e-02 5.19911504e-02
  1.10619469e-03 6.63716814e-03 4.42477876e-03 6.63716814e-03
  9.81308411e-03 2.21238938e-03 3.31858407e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 6.47186147e-01 1.75050302e-01
  3.27967807e-01 3.22434608e-01 1.30081301e-02 8.04828974e-03
  1.00603622e-03 1.50905433e-02 7.48129676e-03 1.10663984e-02
  2.89719626e-02 2.01207243e-03 4.70957614e-03 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.64069264e-01
  7.48917749e-01 5.49783550e-01 3.46320346e-02 1.29870130e-02
  1.51515152e-02 2.59740260e-02 3.24675325e-02 4.32900433e-03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.80748214e-01 2.76640978e-01 1.24661247e-02 2.06327373e-03
  0.00000000e+00 2.83687943e-03 3.69908562e-02 6.64363540e-03
  0.00000000e+00 0.00000000e+0

In [107]:
integrated_2

AnnData object with n_obs × n_vars = 32472 × 15148
    obs: 'dataset', 'location', 'nGene', 'nUMI', 'patientGroup', 'percent.mito', 'protocol', 'sanger_type', 'size_factors', 'sampling_method', 'batch', 'cell_type', 'donor'
    layers: 'counts'

In [35]:
from scgen import SCGEN
SCGEN.setup_anndata(adata, batch_key=batch_key, labels_key=label_key) 
model = SCGEN(adata)
model.train(max_epochs=1, batch_size=640, early_stopping=True)
corrected_adata = model.batch_removal(adata)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\lklei\anaconda3\envs\scib\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\lklei\anaconda3\envs\scib\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 1/1: 100%|██████████| 1/1 [00:16<00:00, 16.23s/it, v_num=1, train_loss_step=349, train_loss_epoch=567]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 1/1: 100%|██████████| 1/1 [00:16<00:00, 16.23s/it, v_num=1, train_loss_step=349, train_loss_epoch=567]


AttributeError: 'NoneType' object has no attribute 'sqrt'

In [32]:
import sys
#if branch is stable, will install via pypi, else will install from source
branch = "stable"
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB and branch == "stable":
    !pip install --quiet scgen[tutorials]
elif IN_COLAB and branch != "stable":
    !pip install --quiet --upgrade jsonschema
    !pip install --quiet git+https://github.com/theislab/scgen@$branch#egg=scgen[tutorials]